In [31]:
import pandas as pd 
import numpy as np
from tabulate import tabulate
data = pd.read_csv("./outputs/blocks-world-ori.csv")

In [32]:
data.head(5)

,Percent,Real_Goal,Time,Cost,Prob,Results
0,10,0,267520804,51/50/50/50/51/50/51/51/50/51/50/50/50/51/50/5...,0.04699862381471914/0.04792925952121189/0.0479...,1/2/3/5/8/10/11/12/14/15/16/18/19/20/
1,10,1,44326438,51/50/51/51/51/51/51/51/51/51/51/51/51/51/51/5...,0.04752941465940805/0.04847056073562341/0.0475...,1/18/
2,10,0,56766524,54/54/53/54/53/54/53/50/53/53/53/54/53/51/51/5...,0.04877251016211816/0.04877251016211816/0.0497...,7/19/
3,10,16,38777939,51/51/51/51/51/51/51/51/51/51/51/51/51/51/51/5...,0.05/0.05/0.05/0.05/0.05/0.05/0.05/0.05/0.05/0...,0/1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18...
4,10,16,54296119,51/51/51/51/51/51/51/51/51/51/51/51/51/51/51/5...,0.05/0.05/0.05/0.05/0.05/0.05/0.05/0.05/0.05/0...,0/1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18...


In [33]:
def precision(stringList, answer):
    goal_count = 0
    found = 0
    for result in stringList:
        if result == str(answer):
            found = 1
        goal_count += 1
    return found/(goal_count-1)

def recall(stringList, answer):
    found = 0
    for result in stringList:
        if result == str(answer):
            found = 1
            break
    return found

def accuracy(total, stringList, answer):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for result in stringList[0:-1]:
        if result == str(answer):
            tp += 1
        else:
            fp += 1
    
    fn = 1 - tp
    
    # total is the number of all goals
    tn = total - tp - fp - fn
    return (tp + tn)/(tn + tp + fp + fn)

In [34]:
length = data.shape[0]
length

868

In [35]:
percent = 10

matrix = []
temp_storage = [0,0,0,0] # p, r, accuracy, time
num = 0

count_goals = 0

for index in range(length):
    new_percent = data.loc[index, "Percent"]
    answer = data.loc[index, "Real_Goal"]
    results = data.loc[index, "Results"].split("/")
    all_candidates = data.loc[index, "Cost"].split("/")
    total = len(all_candidates)-1   # the last one is /
    p = precision(results, answer)
    r = recall(results, answer)
    a = accuracy(total, results, answer)
    
    count_goals += total
    
    time = data.loc[index, "Time"]
    num += 1
    
    if (percent != new_percent):
        temp_storage[0] = temp_storage[0]/num
        temp_storage[1] = temp_storage[1]/num
        temp_storage[2] = temp_storage[2]/num
        temp_storage[3] = temp_storage[3]/num
        
        matrix.append(temp_storage)
        temp_storage = [0,0,0,0]
        num = 0
        percent = new_percent
        
        
    temp_storage[0] += p
    temp_storage[1] += r
    temp_storage[2] += a
    temp_storage[3] += time
    
num += 1
temp_storage[0] = temp_storage[0]/num
temp_storage[1] = temp_storage[1]/num
temp_storage[2] = temp_storage[2]/num
temp_storage[3] = temp_storage[3]/num   
matrix.append(temp_storage)
    
    


In [36]:
table = tabulate(matrix, headers=["p", "r", "a", "t"])
print(table)

        p         r         a            t
---------  --------  --------  -----------
0.0681381  0.623116  0.423247  2.78874e+07
0.154436   0.636364  0.591438  2.55775e+07
0.226117   0.626263  0.676239  2.82001e+07
0.349702   0.580808  0.786448  3.48219e+07
0.82193    0.934211  0.961999  5.30252e+07


In [17]:
## export the table to csv
# A = np.array(matrix)
# np.savetxt("blocksworld.csv", A, delimiter=",")